In [15]:
import openai
import os
import subprocess
import uuid
import re

In [10]:
from openai import OpenAI
client = OpenAI()

In [16]:
def generate_code(prompt: str, model="gpt-4o") -> str:
    system_prompt = "You are a Python code generator. Only output clean, executable Python code inside one code block. No explanations."

    client = OpenAI()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3,
    )

    # Correct access method
    message = response.choices[0].message.content

    # Extract Python code cleanly
    if "```" in message:
        code = message.split("```python")[-1].split("```")[0].strip()
    else:
        code = message.strip()

    return code


import re

def replace_dataset_paths(code: str, dataset_mapping: dict) -> str:
    """
    Replaces placeholder dataset filenames in generated code with actual paths.

    Args:
        code (str): Generated Python code.
        dataset_mapping (dict): Mapping from filename (e.g., 'diabetes.csv') to actual path (e.g., 'datasets/diabetes.csv').

    Returns:
        str: Updated Python code with correct dataset paths.
    """
    pattern = r"pd\.read_csv\(['\"](.*?)['\"]\)"

    def replacer(match):
        filename = match.group(1)
        actual_path = dataset_mapping.get(filename, filename)  # default to original if not found
        return f"pd.read_csv(r'{actual_path}')"

    new_code = re.sub(pattern, replacer, code)
    return new_code


def save_code_to_file(code: str, folder="generated") -> str:
    """Saves code to a uniquely named .py file."""
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, f"code_{uuid.uuid4().hex[:8]}.py")
    with open(file_path, "w") as f:
        f.write(code)
    return file_path


def execute_code_file(file_path: str) -> str:
    """Executes the Python file and captures output."""
    try:
        result = subprocess.run(["python", file_path], capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"❌ Error while executing the script:\n{e.stderr}"

In [20]:
if __name__ == "__main__":
    # Example user prompt
    user_instruction = input("🧠 What Python task would you like me to run?\n")

    print("\n🤖 Generating code...")
    code = generate_code(user_instruction)
    print("\n✅ Code generated:\n")
    print(code)

    file_path = save_code_to_file(code)
    print(f"\n💾 Code saved to: {file_path}")

    print("\n⚙️ Running your code...\n")
    output = execute_code_file(file_path)
    print("\n📤 Output:\n")
    print(output)

🧠 What Python task would you like me to run?
build a random forest classifier in python to predict diabetes in patients

🤖 Generating code...

✅ Code generated:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_diabetes

# Load the diabetes dataset
data = load_diabetes(as_frame=True)
df = data.frame

# Assuming the target variable is 'target' and features are all other columns
X = df.drop(columns='target')
y = df['target']

# Binarize the target variable for classification (e.g., threshold at median)
y_binarized = (y > y.median()).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binarized, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train 